# Ag-Analytics® - Boundary Service API 

It is not uncommon for more than one crop to be grown on a CLU. These CLU boundaries are derived from the last publicly available distribution from 2008. A single CLU is approximately interpreted as a “field”. A Common Land Unit (CLU) is the smallest unit of land that has a permanent, contiguous boundary, a common land cover and land management, a common owner and a common producer in agricultural land associated with USDA farm programs.

The Ag-Analytics Field Boundary API provides a service which a user can pass an extent (bounding box) and retrieve field boundaries in geojson. To our knowledge, this is the only CLU field boundary data service in the market. It is a frequently requested dataset and useful for researchers who seek pre-made field boundaries in order to conduct representative analyses, as well as other apps that wish to serve ‘starter’ field boundaries.

### GET Request Parameters
Note that: in the Boundary Service API, we only set up the GET request. 

| Parameter     | Type         | Required? | Default    | Option                                                            | Description                                                                                                                                                              |
|---------------|--------------|-----------|------------|-------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Geometry      | GeoJSON/JSON | Yes       | 'envelope' | 'envelope','point', 'line','polygon'                              | The AOI used for searching boundaries. It can be a point(x,y coordinate pairs), a line or polygon in GeoJSON, or an envelope(bounding box with xmin, xmax, ymin, ymax).  |
| Geometry_type | String       | Yes       | -          | -                                                                 | The geometry type of the geometry passed in the request.                                                                                                                 |

In [16]:
# required libraries
import requests
import json

## GET request examples

#### 0.Headers

In [17]:
# Basic Header Pattern.
headers=None
# Header for using a subscription key.
# headers={'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

#### 1.Search CLU with a polygon in geojson

In [18]:
polygon_geojson = '{"geometryType": "esriGeometryPolygon", "features": [{"geometry": {"rings": [[[-91.53900737389114, 34.55980772414722], [-91.53894861128936, 34.560481053757655], [-91.53892673258258, 34.56192576336332], [-91.53889249809026, 34.56325987524309], [-91.5388720178293, 34.564625711607334], [-91.53882557593954, 34.566462376337654], [-91.53913423046032, 34.566538207172584], [-91.53917447152463, 34.566730301462314], [-91.53962869840569, 34.566740245266146], [-91.54305890694297, 34.56681730907161], [-91.54305614512498, 34.566585087032195], [-91.54304565183531, 34.56613185570063], [-91.54306887772651, 34.56417543544802], [-91.54310131717199, 34.562940478320456], [-91.54309762365637, 34.56170736210515], [-91.54308258968973, 34.561319449433086], [-91.54285584362225, 34.56111836821816], [-91.54218857094577, 34.56094358497853], [-91.54135688960872, 34.560665392294084], [-91.54036096688509, 34.56031309277506], [-91.53965887605625, 34.560065743239534], [-91.53900737389114, 34.55980772414722]]], "spatialReference": {"wkid": 4326}}}]}'
Geometry_type = 'polygon'
params = {
    "Geometry":polygon_geojson,
    "Geometry_type":Geometry_type}

#### 2.Search CLU with an envelope(bounding box)

In [27]:
envelope = '{"xmin": -101.79931651625, "ymin": 37.45741810262938, "xmax": -101.70931640625, "ymax": 37.45751810262938,"spatialReference": {"wkid": 4326}}'
Geometry_type = 'envelope'
params = {
    "Geometry":envelope,
    "Geometry_type":Geometry_type}

#### 3.Search CLU  with a point(x,y coords pair)

In [20]:
aoi_point = '{"x":-91.53900737389114,"y": 34.55980772414722}'
Geometry_type = 'point'
params = {
    "Geometry":envelope,
    "Geometry_type":Geometry_type}

## GET request function

In [25]:
def Get_Boundaries(params, headers):
    API_URL = "https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/Boundary_Service_API"
    response = requests.get(API_URL, params = params,headers = headers)
    if response.status_code == 200:
        print('----- API Call Succeed ----')
        print(response.text)
        return response.json()
    else:
        print('----- API Call Failed ----')
        print(response.text)
        
    

In [29]:
# Call Boundary Service API
resp = Get_Boundaries(params, headers)

----- API Call Succeed ----
{"msg":"Found 13 results","results":{"crs":{"properties":{"name":"EPSG:4326"},"type":"name"},"features":[{"geometry":{"coordinates":[[[-101.78961088123714,37.461310953138025],[-101.78961144511209,37.461294494645244],[-101.7985410286455,37.46140816895189],[-101.79850997325661,37.457531276147506],[-101.79848254843085,37.454104472443476],[-101.79841264772546,37.4541947418939],[-101.7984059145013,37.455092100719924],[-101.79821392813022,37.455179046276044],[-101.79774520237913,37.45523592749612],[-101.79760400342201,37.45576111268525],[-101.79763844385906,37.456684961042185],[-101.79771076104277,37.45954879903792],[-101.7975984024443,37.45961554672016],[-101.79326633037681,37.45955331003728],[-101.79308920530252,37.45973144684882],[-101.79312209710707,37.46123210417829],[-101.78959380401079,37.46120737102342],[-101.78959478876845,37.4612834087024],[-101.7895951368061,37.46131071211971],[-101.78961088123714,37.461310953138025]]],"type":"Polygon"},"id":8180685,"pr

## Visualize result boundaries

In [21]:
# required library
import folium
import mplleaflet

In [30]:
#Display parcels on map
result_geojson = resp['results']
zoom_point=result_geojson['features'][0]['geometry']['coordinates'][0][0]
zoom_point
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in result_geojson['features']:
    folium.GeoJson(shape).add_to(m)
m